<a href="https://colab.research.google.com/github/murilofarias10/Python/blob/main/New_PROMPT_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install -q huggingface_hub

In [63]:
import os
from huggingface_hub import InferenceClient

os.environ["HF_TOKEN"]="x"

#client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct")
#client = InferenceClient("mistralai/Mistral-7B-Instruct-v0.2")
client = InferenceClient("deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")



In [64]:
output = client.text_generation(
    "The capital of Brazil is",
    max_new_tokens=100,
)

print(output)

 Brasília. It was designed by urban planner Lúcio Costa and architect Oscar Niemeyer. The city is known for its modernist architecture and unique urban planning. The main square is called the Three Powers Plaza, which houses the executive, legislative, and judicial branches of the government. The city is located in the central-west region of Brazil and was built to serve as the new capital in 1960, replacing Rio de Janeiro. The area around Brasília is known for its cerr


In [65]:
SYSTEM_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}
example use :
```
{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:
```
$JSON_BLOB
```
Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/Observation can repeat N times, you should take several steps when needed. The $JSON_BLOB must be formatted as markdown and only use a SINGLE action at a time.)

You must always end your output with the following format:

Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer. """

In [68]:
# Since we are running the "text_generation", we need to add the right special tokens.
prompt=f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{SYSTEM_PROMPT}
<|eot_id|><|start_header_id|>user<|end_header_id|>
The capital of Brazil is ?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [69]:
output = client.text_generation(
    prompt,
    max_new_tokens=200,
)

print(output)

Okay, so I need to figure out the capital of Brazil. Hmm, I remember that Brazil is a country in South America, but I'm not exactly sure about its capital. I think it's not Rio de Janeiro because I've heard that's a major city but not the capital anymore. Maybe it's Brasília? I think that's the one. But wait, how can I be certain? I should probably look it up to make sure I'm not mixing things up. 

Wait, the user mentioned that I can use a tool called get_weather, but that's for weather information. That doesn't seem helpful for finding a capital city. Maybe I should just rely on my existing knowledge. I'm pretty confident that Brasília is the capital, but I want to double-check. 

Alternatively, maybe I can think about when Brazil moved its capital. I recall that it was moved from Rio de Janeiro to Brasília in the mid-20th century, around the 19


In [70]:
# The answer was hallucinated by the model. We need to stop to actually execute the function!
output = client.text_generation(
    prompt,
    max_new_tokens=200,
    stop=["Observation:"] # Let's stop before any actual function is called
)

print(output)

Okay, so I need to figure out the capital of Brazil. Hmm, I remember that Brazil is a country in South America, but I'm not exactly sure about its capital. I think it's not Rio de Janeiro because I've heard that's a major city but not the capital anymore. Maybe it's Brasília? I think that's the one. But wait, how can I be certain? I should probably look it up to make sure I'm not mixing things up. 

Wait, the user mentioned that I can use a tool called get_weather, but that's for weather information, not for capitals. So maybe I can't use that tool for this question. That means I have to rely on my existing knowledge. Let me think again. Brazil's capital is Brasília, right? It was built in the mid-20th century to be the new capital, moving away from Rio de Janeiro. Yeah, that sounds familiar from history classes. So I'm pretty confident that


In [71]:
# Dummy function
def get_weather(location):
    return f"the weather in {location} is good in summer. \n"

get_weather('Brazil')

'the weather in Brazil is good in summer. \n'

In [72]:
# Let's concatenate the base prompt, the completion until function execution and the result of the function as an Observation
new_prompt=prompt+output+get_weather('Brazil')
print(new_prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Answer the following questions as best you can. You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
get_weather: Get the current weather in a given location, args: {"location": {"type": "string"}}
example use :
```
{{
  "action": "get_weather",
  "action_input": {"location": "New York"}
}}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time in this format:
Action:
```
$JSON_BLOB
```
Observation: the result of the action. This Observation is unique, complete, and the source of truth.
... (this Thought/Action/

In [73]:
final_output = client.text_generation(
    new_prompt,
    max_new_tokens=200,
)

print(final_output)

</think>

The capital of Brazil is Brasília.
